### **Zadanie: Refaktoryzacja kodu pod kątem SOLID**

Masz kod, który działa, ale łamie kilka zasad SOLID. Twoim zadaniem jest poprawienie go tak, aby przestrzegał tych zasad. Kod jest prosty, ale pełen potencjalnych problemów związanych z rozszerzalnością i utrzymaniem.

---

#### **Zastany kod:**
```python
class Order:
    def __init__(self, items, total_price):
        self.items = items
        self.total_price = total_price

    def calculate_discount(self, customer_type):
        if customer_type == "regular":
            return self.total_price * 0.9
        elif customer_type == "vip":
            return self.total_price * 0.8
        else:
            return self.total_price

    def generate_invoice(self):
        invoice = f"Invoice:\nItems: {self.items}\nTotal: {self.total_price}"
        with open("invoice.txt", "w") as file:
            file.write(invoice)
        return invoice

    def send_email(self, email):
        invoice = self.generate_invoice()
        print(f"Sending email to {email} with the following invoice:\n{invoice}")
```

---

#### **Twoje zadanie:**

1. **Zidentyfikuj problemy:**
   - Które zasady SOLID są łamane?
   - Dlaczego kod jest trudny do rozszerzenia i utrzymania?

2. **Zrefaktoryzuj kod:**
   - Zastosuj odpowiednie zasady SOLID.
   - Upewnij się, że kod jest elastyczny, łatwy w testowaniu i zgodny z dobrymi praktykami.

---

#### **Wynik oczekiwany:**

1. Kod powinien być podzielony na mniejsze, bardziej odpowiedzialne klasy.
2. Logika rabatów, generowania faktur i wysyłania e-maili powinna być oddzielona.
3. Nowe typy rabatów (np. „student”) powinny być łatwe do dodania bez modyfikowania istniejącego kodu.




In [25]:
from abc import ABC, abstractmethod
from dataclasses import dataclass

REGULAR_DISCOUNT = 0.9
VIP_DISCOUNT = 0.8

@dataclass
class Order:
    items: list
    total_price: float

class Discount(ABC):
    @abstractmethod
    def apply_discount(self, total_price): ...

class RegularCustomerDiscount(Discount):
    def apply_discount(self, total_price): 
        return total_price * REGULAR_DISCOUNT

class VIPCustomerDiscount(Discount):
    def apply_discount(self, total_price): 
        return total_price * VIP_DISCOUNT

class NoDiscount(Discount):
    def apply_discount(self, total_price): 
        return total_price


In [23]:
from textwrap import dedent

class InvoiceGenerator:
    def generate_invoice(self, order: Order) -> str:
        invoice = dedent(f"""
        Invoice:
        Items: {order.items}
        Total: {order.total_price}
        """)
        return invoice


class InvoiceExporter(ABC):

    @abstractmethod
    def export(self, filename: str, invoice: str):
        ...

class TextInvoiceExporter(InvoiceExporter):
    def export(self, filename: str, invoice: str):
        with open(filename, "w") as f:
            f.write(invoice)



class Notification(ABC):

    @abstractmethod
    def send(self, invoice: str): ...


class EmailNotification(Notification):
    def send(self, invoice: str): 
        print("wysylam emial ...")

        
class KSFNotification(Notification):
    def send(self, invoice: str): 
        print("wysylam protokolem ksf ...")

In [27]:
class OrderProcessor:
    def __init__(self, discount: Discount, invoice_generator: InvoiceGenerator, notifier: Notification, exporter: InvoiceExporter):
        self.discount = discount
        self.generator = invoice_generator
        self.notifier = notifier
        self.exporter = exporter

    def process_order(self, order):
        order.total_price = self.discount.apply_discount(order.total_price)

        invoice = self.generator.generate_invoice(order)

        self.exporter.export("invoice.txt", invoice)
        self.notifier.send(invoice)
        

In [28]:
order = Order(items=["A", "V"], total_price = 100)

discount = VIPCustomerDiscount()
invoice_generator = InvoiceGenerator()
notifier = EmailNotification()
exporter = TextInvoiceExporter()

processor = OrderProcessor(discount=discount, invoice_generator=invoice_generator, notifier=notifier, exporter=exporter)
processor.process_order(order)

wysylam emial ...


In [ ]:
CUPID